# VacationPy

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# WeatherPy Results Dataframe

In [26]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities = pd.read_csv("../WeatherPy/city_weather.csv", encoding="utf-8")
cities.head()

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,2643587,Lossiemouth,GB,57.7214,-3.2834,59.13,58,75,16.11
1,2155415,New Norfolk,AU,-42.7826,147.0587,46.45,76,99,1.01
2,3874787,Punta Arenas,CL,-53.1500,-70.9167,37.11,80,100,14.56
3,1337610,Thinadhoo,MV,0.5333,72.9333,84.94,75,30,15.50
4,4032243,Vaini,TO,-21.2000,-175.2000,73.56,88,40,4.61


### Humidity Heatmap

In [27]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [28]:
# Using Lat and Lng as locations:
locations = cities[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()

In [29]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)


# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = cities[cities["Clouds"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Temperature"] > 70) & (cloud_df["Temperature"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
24,3985710,Cabo San Lucas,MX,22.8909,-109.9124,77.14,67,0,3.44
48,2063042,Port Hedland,AU,-20.3167,118.5667,73.45,40,0,9.22
112,5546220,Saint George,US,37.1041,-113.5841,76.23,19,0,1.99
164,2269594,Cascais,PT,38.6979,-9.4215,75.49,37,0,7.00
224,1260206,Pāsighāt,IN,28.0667,95.3333,75.54,69,0,4.32
295,1018725,Bloemfontein,ZA,-29.1211,26.2140,71.08,26,0,4.61
371,2068110,Kununurra,AU,-15.7667,128.7333,78.78,29,0,9.22
442,3855075,General Pico,AR,-35.6566,-63.7568,74.52,30,0,7.05
545,4013039,Concepción del Oro,MX,24.6333,-101.4167,74.52,25,0,2.04
549,3901547,Villamontes,BO,-21.2500,-63.5000,72.84,43,0,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df 

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
24,3985710,Cabo San Lucas,MX,22.8909,-109.9124,77.14,67,0,3.44,Medano Hotel and Suites
48,2063042,Port Hedland,AU,-20.3167,118.5667,73.45,40,0,9.22,Hedland Hotel
112,5546220,Saint George,US,37.1041,-113.5841,76.23,19,0,1.99,Hilton Garden Inn St. George
164,2269594,Cascais,PT,38.6979,-9.4215,75.49,37,0,7.00,"InterContinental Cascais-Estoril, an IHG Hotel"
224,1260206,Pāsighāt,IN,28.0667,95.3333,75.54,69,0,4.32,Donyi Hango Camp Homestay And Resort
295,1018725,Bloemfontein,ZA,-29.1211,26.2140,71.08,26,0,4.61,Protea Hotel by Marriott Bloemfontein
371,2068110,Kununurra,AU,-15.7667,128.7333,78.78,29,0,9.22,The Kimberley Grande Resort
442,3855075,General Pico,AR,-35.6566,-63.7568,74.52,30,0,7.05,Apart Hotel Aires de La Pampa
545,4013039,Concepción del Oro,MX,24.6333,-101.4167,74.52,25,0,2.04,Mesón del Poeta
549,3901547,Villamontes,BO,-21.2500,-63.5000,72.84,43,0,5.75,Hotel Ecológico Gota del Chaco


In [32]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
24,3985710,Cabo San Lucas,MX,22.8909,-109.9124,77.14,67,0,3.44,Medano Hotel and Suites
48,2063042,Port Hedland,AU,-20.3167,118.5667,73.45,40,0,9.22,Hedland Hotel
112,5546220,Saint George,US,37.1041,-113.5841,76.23,19,0,1.99,Hilton Garden Inn St. George
164,2269594,Cascais,PT,38.6979,-9.4215,75.49,37,0,7.00,"InterContinental Cascais-Estoril, an IHG Hotel"
224,1260206,Pāsighāt,IN,28.0667,95.3333,75.54,69,0,4.32,Donyi Hango Camp Homestay And Resort
295,1018725,Bloemfontein,ZA,-29.1211,26.2140,71.08,26,0,4.61,Protea Hotel by Marriott Bloemfontein
371,2068110,Kununurra,AU,-15.7667,128.7333,78.78,29,0,9.22,The Kimberley Grande Resort
442,3855075,General Pico,AR,-35.6566,-63.7568,74.52,30,0,7.05,Apart Hotel Aires de La Pampa
545,4013039,Concepción del Oro,MX,24.6333,-101.4167,74.52,25,0,2.04,Mesón del Poeta
549,3901547,Villamontes,BO,-21.2500,-63.5000,72.84,43,0,5.75,Hotel Ecológico Gota del Chaco


In [33]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

10

In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Checking hotel info:
len(hotel_info)

10

In [37]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))